In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time

torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset)
    
def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    offset = random.randint(-5,5)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist))
        signal2.append(random.choice(dist))
    return signal1, rotateList(signal2,offset), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    for i in range(num):
        sig1, sig2, offset = createsignal(limit = limit)
        train_data = np.stack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data.reshape(-1,1)).view(-1)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
            train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq
# print(createsignal())

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

Creating Data and training for normalized data

In [ ]:
train_input = createsignals(num=10000, normalize = True)
test_input = createsignals()
# training_length = int(len(input)*(.8))
# train_input = input[:training_length]
# val_input = input[training_length:] 

In [ ]:
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 150
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/150
----------
Epoch 1 Loss 8.212363118856208
Epoch 2/150
----------
Epoch 2 Loss 8.192705037938058
Epoch 3/150
----------
Epoch 3 Loss 8.190670896953717
Epoch 4/150
----------
Epoch 4 Loss 8.189504792851954
Epoch 5/150
----------
Epoch 5 Loss 8.188716535983234
Epoch 6/150
----------
Epoch 6 Loss 8.188116573823988
Epoch 7/150
----------
Epoch 7 Loss 8.187628107652069
Epoch 8/150
----------
Epoch 8 Loss 8.187224545290322
Epoch 9/150
----------
Epoch 9 Loss 8.186880488904565
Epoch 10/150
----------
Epoch 10 Loss 8.186573987582326
Epoch 11/150
----------
Epoch 11 Loss 8.186301475220173
Epoch 12/150
----------
Epoch 12 Loss 8.18605458612442
Epoch 13/150
----------
Epoch 13 Loss 8.185833382226527
Epoch 14/150
----------
Epoch 14 Loss 8.185632696180791
Epoch 15/150
----------
Epoch 15 Loss 8.18544627309218
Epoch 16/150
----------
Epoch 16 Loss 8.185271648608149
Epoch 17/150
----------
Epoch 17 Loss 8.185105155174433
Epoch 18/150
----------
Epoch 18 Loss 8.184945206344128
Epoch 19/150

In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr

**Create data and train without normalization**

In [ ]:
train_input = createsignals(num=1000, normalize = False)
test_input = createsignals()
# training_length = int(len(input)*(.8))
# train_input = input[:training_length]
# val_input = input[training_length:] 

In [ ]:
model = LSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
epochs = 150
model.train()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    model.zero_grad()
    for seq, labels in train_input:

        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     print(f'epoch: {i:3} loss: {loss.item():10.8f}')

# print(f'epoch: {i:3} loss: {loss.item():10.10f}')

Epoch 1/150
----------
Epoch 1 Loss 8.084122549632466
Epoch 2/150
----------
Epoch 2 Loss 7.9966126870810985
Epoch 3/150
----------
Epoch 3 Loss 7.98839268996939
Epoch 4/150
----------
Epoch 4 Loss 7.981902225129306
Epoch 5/150
----------
Epoch 5 Loss 7.97621142341569
Epoch 6/150
----------
Epoch 6 Loss 7.970912792820483
Epoch 7/150
----------
Epoch 7 Loss 7.965781392733566
Epoch 8/150
----------
Epoch 8 Loss 7.960704465313232
Epoch 9/150
----------
Epoch 9 Loss 7.955567259442486
Epoch 10/150
----------
Epoch 10 Loss 7.950213516256481
Epoch 11/150
----------
Epoch 11 Loss 7.944430255983956
Epoch 12/150
----------
Epoch 12 Loss 7.938025259397225
Epoch 13/150
----------
Epoch 13 Loss 7.930949869328121
Epoch 14/150
----------
Epoch 14 Loss 7.9232085605362474
Epoch 15/150
----------
Epoch 15 Loss 7.914816875344386
Epoch 16/150
----------
Epoch 16 Loss 7.905895181384687
Epoch 17/150
----------
Epoch 17 Loss 7.8966156308078785
Epoch 18/150
----------
Epoch 18 Loss 7.887105986985176
Epoch 19/

In [ ]:
epochs = 1
model.eval()
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    print('-' * 10)
    epoch_loss = 0.0
    running_loss = 0.0
    # model.zero_grad()
    for seq, labels in test_input:

        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                 torch.zeros(1, 1, model.hidden_layer_size))
        # # seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq)#.to(device)
        print(f"Offset {labels.data[0]}, Prediction {y_pred.data[0]}")
        # optimizer.zero_grad()
        loss = criterion(y_pred, labels)
        # loss.backward()
        # optimizer.step()
        epoch_loss += y_pred.shape[0] * loss.item()

        # print statistics
        # running_loss += loss.item()
        # if i % 25 == 1:    
        #     print('[%d] loss: %.3f' %
        #           (i + 1, running_loss / 25))
        #     running_loss = 0.0
    print("Epoch {} Loss {}".format(epoch+1, epoch_loss / len(train_input)))

    # if i%25 == 1:
#     pr

Epoch 1/1
----------
Offset -5.0, Prediction -1.9137938022613525
Offset 3.0, Prediction 1.1839913129806519
Offset -1.0, Prediction -2.2383506298065186
Offset 0.0, Prediction -3.61566424369812
Offset 0.0, Prediction -4.860353469848633
Offset -5.0, Prediction 0.7327643632888794
Offset -5.0, Prediction -1.5109379291534424
Offset 2.0, Prediction -0.26550501585006714
Offset -5.0, Prediction -2.0336103439331055
Offset -1.0, Prediction -0.4884141981601715
Offset -2.0, Prediction -4.797237873077393
Offset 2.0, Prediction -2.3178112506866455
Offset -5.0, Prediction -5.90170955657959
Offset -3.0, Prediction 2.380894899368286
Offset -1.0, Prediction -0.9219677448272705
Offset -4.0, Prediction 2.858970880508423
Offset -3.0, Prediction -2.450437307357788
Offset 0.0, Prediction 0.7322994470596313
Offset 4.0, Prediction -3.298802375793457
Offset -4.0, Prediction -2.8143608570098877
Offset -2.0, Prediction -2.1763675212860107
Offset -4.0, Prediction -3.4847307205200195
Offset -3.0, Prediction -3.48069